In [1]:
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Flatten,InputLayer

In [2]:
train = pd.read_csv('train.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
train.shape

(1460, 81)

In [5]:
train_X,train_Y = train.iloc[:,:train.shape[1]-1],train.iloc[:,train.shape[1]-1]

In [7]:
train_Y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [8]:
# taking the categorical columns from trainx to peform one hot encoding
categoricals = train_X.loc[:,train_X.dtypes == 'O'].columns

In [9]:
len(categoricals)

43

In [10]:
# One hot encoding doesnt work with NaN columns so filling missing values with mode
cat_features = train_X.loc[:,categoricals]
cat_features = cat_features.fillna(cat_features.mode().iloc[0,:])

In [12]:
# One hot encoding these features
ohe = OneHotEncoder(handle_unknown='ignore')
res = ohe.fit_transform(cat_features)

In [17]:
res = res.toarray()

In [16]:
cols = np.array([])
for i in range(cat_features.shape[1]):
    cols = np.concatenate((cols, categoricals[i] + '_' + np.sort(cat_features.iloc[:,i].unique())))


In [18]:
cat = pd.DataFrame(res,columns=cols)

In [19]:
cat.shape

(1460, 251)

In [22]:
# dropping original categorical features form data
train_X = train_X.drop(categoricals, axis = 1)

In [23]:
# concatenate one hot encoded columns to the train dataset
train_X = pd.concat([train_X,cat], axis = 1)

In [24]:
train_X.shape

(1460, 288)

In [25]:
# filling the Numerical columns Nan with their median
train_X.fillna(train_X.median(), inplace=True)

In [27]:
# normalising training features
scalar = MinMaxScaler()
norm_train = pd.DataFrame(scalar.fit_transform(train_X),columns = train_X.columns)

In [28]:
# normalising training target
scalar_target = MinMaxScaler()
train_Y= scalar_target.fit_transform(train_Y.values.reshape(-1,1))

In [29]:
# defining the model network
model = Sequential([
    Dense(norm_train.shape[1], input_dim=norm_train.shape[1],activation='sigmoid'),
    Dense(units=norm_train.shape[1]//2,activation='sigmoid'),
    Dense(units=1,activation='softmax'),
])


C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [31]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 288)                 │          83,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 144)                 │          41,616 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             145 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 124,993 (488.25 KB)

 Trainable params: 124,993 (488.25 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
model.compile(optimizer='sgd',loss = 'mean_squared_error',)
model.fit(train_X,train_Y,batch_size=512,epochs=20,verbose=1,validation_split=0.2)

Epoch 1/20


C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\ops\nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step - loss: 0.6471 - val_loss: 0.6456
Epoch 2/20


C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\ops\nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.6494 - val_loss: 0.6456
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.6489 - val_loss: 0.6456
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.6470 - val_loss: 0.6456
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 0.6499 - val_loss: 0.6456
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.6467 - val_loss: 0.6456
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.6496 - val_loss: 0.6456
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.6483 - val_loss: 0.6456
Epoch 9/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.6465 - val_loss: 0.6456
Epoch 10/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.6465 - val_loss: 0.6456
Epoch 11/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.6477 - val_loss: 0.6456
Epoch 12/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.6467 - val_loss: 0.6456
Epoch 13/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.6488 - val_loss: 0.6456
Epoch 14/2

## HyperParameter tuning

In [ ]:
Outcomes:
activation: relu, relu , linear

In [33]:
import os
import re
from tensorflow.python.keras import utils

In [34]:
input_num_units = 288
hidden_num_units = 144
output_num_units = 1
epochs = 50
batch_size = 500

### 1. Activation function

In [37]:
def models_with_different_activation_functions(list_of_activation_functions):
    for i in range(len(list_of_activation_functions)):
        # Defining the network
        model_2 = Sequential([
            Dense(norm_train.shape[1],input_dim = norm_train.shape[1], activation='sigmoid'),
            Dense(units = norm_train.shape[1]//2, activation='sigmoid'),
            Dense(units =1,activation='softmax'),
        ])
        model_2.compile(optimizer='sgd',loss='mean_squared_error')
        # Traning the model and writing log files for TensorBoard in distinct directories                
        logdir = r'activation\\' + list_of_activation_functions[i] # Each log file needs to be written in a distinct directory. (Mandatory)
        
        # Writing graph will take time. Hence, keeping it False.
        cb = keras.callbacks.TensorBoard(log_dir=logdir, write_graph=False)         
        print('Building model using '+ list_of_activation_functions[i] + ' activation function')

        # Training the model
        history= model_2.fit(train_X,train_Y,batch_size=batch_size,epochs=epochs,verbose=1,validation_split=0.2)
        print('model built successfully')
        print('')
# List of activation functions
lis = ['linear', 'sigmoid', 'tanh', 'relu', 'softmax']

#calling the function
models_with_different_activation_functions(act)        

Building model using linear activation function
Epoch 1/50


C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\ops\nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step - loss: 0.6474 - val_loss: 0.6456
Epoch 2/50


C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\ops\nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.6463 - val_loss: 0.6456
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.6438 - val_loss: 0.6456
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.6505 - val_loss: 0.6456
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.6460 - val_loss: 0.6456
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.6507 - val_loss: 0.6456
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.6492 - val_loss: 0.6456
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.6471 - val_loss: 0.6456
Epoch 9/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.6488 - val_loss: 0.6456
Epoch 10/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.6491 - val_loss: 0.6456
Epoch 11/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.6467 - val_loss: 0.6456
Epoch 12/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.6498 - val_loss: 0.6456
Epoch 13/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.6489 - val_loss: 0.6456
Epoch 14/50

...

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.6488 - val_loss: 0.6456
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.6464 - val_loss: 0.6456
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.6497 - val_loss: 0.6456
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.6483 - val_loss: 0.6456
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.6494 - val_loss: 0.6456
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.6499 - val_loss: 0.6456
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.6504 - val_loss: 0.6456
Epoch 9/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.6470 - val_loss: 0.6456
Epoch 10/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.6479 - val_loss: 0.6456
Epoch 11/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.6500 - val_loss: 0.6456
Epoch 12/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.6501 - val_loss: 0.6456
Epoch 13/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.6478 - val_loss: 0.6456
Epoch 14/50

C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\ops\nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: 0.6466 - val_loss: 0.6456
Epoch 2/50
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.6562

C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\ops\nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.6505 - val_loss: 0.6456
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.6472 - val_loss: 0.6456
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.6453 - val_loss: 0.6456
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.6485 - val_loss: 0.6456
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.6476 - val_loss: 0.6456
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.6483 - val_loss: 0.6456
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.6491 - val_loss: 0.6456
Epoch 9/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.6493 - val_loss: 0.6456
Epoch 10/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.6476 - val_loss: 0.6456
Epoch 11/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.6480 - val_loss: 0.6456
Epoch 12/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.6507 - val_loss: 0.6456
Epoch 13/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.6473 - val_loss: 0.6456
Epoch 14/50